<a href="https://colab.research.google.com/github/rammiahj/GitTutorial/blob/master/pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image, ImageDraw
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import torch.optim as optim
import torchvision
import glob


In [ ]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
#Dataloader

#Path for training and testing directory
train_img='/content/drive/MyDrive/ASLO/training'
test_img='/content/drive/MyDrive/ASLO/testing'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_img,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_img,transform=transformer),
    batch_size=32, shuffle=True
)
print(f"Train Classes : {len(train_img)}")
print(f"Test Classes : {len(test_img)}")

Train Classes : 36
Test Classes : 35


In [ ]:
#calculating the size of training and testing images
train_count=len(glob.glob(train_img+'/**/*.tif'))
test_count=len(glob.glob(test_img+'/**/*.tif'))

print(train_count,test_count)

3300 3310


In [ ]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#print device
print('Using device : ',device)

Using device :  cpu


In [ ]:
!wget https://download.pytorch.org/models/resnet50-0676ba61.pth 
model = models.resnet50()
model.load_state_dict(torch.load('resnet50-0676ba61.pth'))

for param in model.parameters(): #Freezing the model
  param.requires_grad = False

model.fc = torch.nn.Linear(in_features=2048, out_features=22, bias=True)
#model.fc.weight.requires_grad=True

model=model.to(device)

--2021-07-08 03:02:33--  https://download.pytorch.org/models/resnet50-0676ba61.pth
Resolving download.pytorch.org (download.pytorch.org)... 13.226.36.99, 13.226.36.97, 13.226.36.52, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.226.36.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102530333 (98M) [application/x-www-form-urlencoded]
Saving to: ‘resnet50-0676ba61.pth’

resnet50-0676ba61.p 100%[===================>]  97.78M  21.6MB/s    in 4.4s    

2021-07-08 03:02:38 (22.2 MB/s) - ‘resnet50-0676ba61.pth’ saved [102530333/102530333]



In [ ]:
#Optimizer and Loss function
criterian = nn.CrossEntropyLoss() #CrossEntropyLoss for classification problem
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5) #lr and weight_decay are hyper parameters 

In [ ]:
#Model training and saving best model

best_accuracy=0.0

for epoch in range(20):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=criterian(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count

    # Evaluation on testing dataset
    #with torch.no_grad():
    model.eval()  
    test_accuracy=0.0

    for i, (images,labels) in enumerate(test_loader):
      if torch.cuda.is_available():
          images=Variable(images.cpu())
          labels=Variable(labels.cpu())
            
      outputs=model(images)
      _,prediction=torch.max(outputs.data,1)
      test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

print("Finished Training and Testing")

Epoch: 0 Train Loss: tensor(0.8221) Train Accuracy: 0.7981818181818182 Test Accuracy: 0.793655589123867
Epoch: 1 Train Loss: tensor(0.5585) Train Accuracy: 0.8490909090909091 Test Accuracy: 0.8492447129909365
Epoch: 2 Train Loss: tensor(0.4571) Train Accuracy: 0.8815151515151515 Test Accuracy: 0.8531722054380665
Epoch: 3 Train Loss: tensor(0.3868) Train Accuracy: 0.8918181818181818 Test Accuracy: 0.8525679758308157
Epoch: 4 Train Loss: tensor(0.3471) Train Accuracy: 0.9012121212121212 Test Accuracy: 0.8577039274924472
Epoch: 5 Train Loss: tensor(0.3221) Train Accuracy: 0.9112121212121213 Test Accuracy: 0.8661631419939577
Epoch: 6 Train Loss: tensor(0.2852) Train Accuracy: 0.9209090909090909 Test Accuracy: 0.8661631419939577
Epoch: 7 Train Loss: tensor(0.2654) Train Accuracy: 0.9221212121212121 Test Accuracy: 0.8676737160120845
Epoch: 8 Train Loss: tensor(0.2405) Train Accuracy: 0.9345454545454546 Test Accuracy: 0.8716012084592145
Epoch: 9 Train Loss: tensor(0.2309) Train Accuracy: 0.93